# Do Palace Merging

In [ ]:
import pickle
from collections import deque
from pathlib import Path
from typing import Any, Dict, List, Optional, Set, Tuple

import graphviz as gv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from gtsam import Pose3

from gtsfm.graph_partitioner.metis_partitioner import MetisPartitioner
from gtsfm.products.cluster_tree import ClusterTree
from gtsfm.products.visibility_graph import (VisibilityGraph,
                                             visibility_graph_keys)
from gtsfm.utils.io import load_poses, read_images_txt, save_poses

PALACE = Path("../tests/data/palace")

In [ ]:
df = pd.read_csv(PALACE / 'visibility_graph.csv')
graph : VisibilityGraph = list(zip(df["i"], df["j"]))

In [ ]:
print("Number of edges in visibility graph:", len(graph))
print("Number of keys:", len(visibility_graph_keys(graph)))

In [ ]:
partitioner = MetisPartitioner()
cluster_tree = partitioner.run(graph)

In [ ]:
with open(PALACE / 'cluster_tree.pkl', 'rb') as f:
    loaded_cluster_tree = pickle.load(f)
print('Loaded cluster_tree from', PALACE / 'cluster_tree.pkl')

In [ ]:
# Load all file names in images path, put them in a sorted list, and create a dictionary from image name to index
images_path = Path("../benchmarks/palace-fine-arts-281/images")
image_files = sorted([f.name for f in images_path.iterdir() if f.is_file()])
image_name_to_index = {name: idx for idx, name in enumerate(image_files)}

In [ ]:
cluster_tree

In [ ]:
# Traverse folders, load all colmap images, and print out their image indices, sorted by relative path and only print relative path
base_dir = Path("../vggt")
images_txt_files = sorted(base_dir.rglob("images.txt"), key=lambda p: str(p.relative_to(base_dir)))
for images_txt_path in images_txt_files:
    colmap_path = images_txt_path.parent
    poses, img_fnames = read_images_txt(str(images_txt_path))
    keys = [image_name_to_index[fname] for fname in img_fnames]
    print(f"{images_txt_path.relative_to(base_dir)} ({len(keys)}): {keys}")

In [ ]:
# Print the number of local keys in each cluster in cluster_tree

def print_local_keys(cluster_tree):
    def traverse(node, name):
        num_keys = len(node.local_keys())
        print(f"{name}: {num_keys} local keys")
        for i, child in enumerate(node.children):
            traverse(child, f"{name}_{i+1}")
    traverse(cluster_tree, "C")

print_local_keys(loaded_cluster_tree)

In [ ]:
# Recursively print number of images in the 'images' subfolder for all folders under ../vggt, sorted by relative path, and print image indices from images.txt if present
base_dir = Path("../vggt")
images_dirs = sorted([p for p in base_dir.rglob("images") if p.is_dir()], key=lambda p: str(p.relative_to(base_dir)))
for images_path in images_dirs:
    num_images = len([f for f in images_path.iterdir() if f.suffix.lower() in {'.jpg', '.jpeg', '.png'}])
    print(f"{images_path.relative_to(base_dir)}: {num_images} images")
    images_txt_path = images_path.parent / "images.txt"
    if images_txt_path.exists():
        poses, img_fnames = read_images_txt(str(images_txt_path))
        print(f"  {images_txt_path.relative_to(base_dir)}:")
        for idx, fname in enumerate(img_fnames):
            print(f"    Index {idx}: {fname}")